# Matrix-Multiplication

## Multiplying two large matrices using Parallel code and then benchmarking it with the serial implementation of the same.


In [1]:
using Pkg
Pkg.add("CUDAnative")
Pkg.add("CuArrays")
Pkg.add("CUDAdrv")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [0]:
using CUDAnative,CuArrays,CUDAdrv,BenchmarkTools,Test

In [4]:
N = 2^11
a = rand(Float32,N,N)
b = rand(Float32,N,N)
parallel_result = zeros(Float32,N,N)

2048×2048 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [5]:
# Function to multiply two matrices using serial code

function serial_mult(a,b,N)
  c = zeros(N,N)
  c = a*b
  return c
end

serial_mult (generic function with 1 method)

In [6]:
serial_result = serial_mult(a,b,N)

2048×2048 Array{Float32,2}:
 500.211  505.209  494.247  500.829  …  497.685  511.218  510.327  511.967
 518.749  514.911  514.287  519.131     517.602  526.828  521.151  526.347
 497.019  502.988  497.434  502.209     498.48   506.109  499.244  512.529
 514.641  516.13   511.534  518.931     511.175  519.687  520.316  519.163
 512.068  517.94   509.921  503.95      507.12   516.948  512.325  520.267
 485.998  502.84   491.221  500.771  …  492.548  501.295  498.827  501.975
 500.93   503.422  496.193  500.245     491.89   505.215  497.834  512.739
 510.457  520.26   509.842  511.498     507.358  519.877  511.89   519.129
 518.471  526.671  508.699  523.33      523.943  528.418  522.436  529.202
 493.97   508.688  495.181  497.276     495.346  508.618  505.536  512.053
 507.439  514.697  503.336  511.782  …  516.754  511.987  516.746  519.88 
 493.011  500.946  498.185  502.382     502.943  514.524  503.264  509.855
 494.976  510.644  498.355  505.626     503.671  501.076  499.159  512.6

In [7]:
d_a = CuArray(a)
d_b = CuArray(b)
d_output = CuArray(parallel_result)

2048×2048 CuArray{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [8]:
# Function to multiply two matrices using serial code

function parallel_mult(d_a,d_b,d_output,N)
  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x 
  idy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
  if(idx <= N && idy <= N)
  sum = 0
  for i = 1:N
    @inbounds sum = sum + d_a[idx,i] * d_b[i,idy]
    end
  @inbounds d_output[idx,idy] = sum
  end
return
end

parallel_mult (generic function with 1 method)

In [0]:
@cuda blocks = (Int(ceil(N/32)),Int(ceil(N/32))) threads = (32,32) parallel_mult(d_a,d_b,d_output,N)

In [10]:
parallel_result = Array(d_output)

2048×2048 Array{Float32,2}:
 500.211  505.209  494.247  500.829  …  497.685  511.219  510.327  511.968
 518.75   514.91   514.286  519.131     517.602  526.828  521.151  526.346
 497.019  502.987  497.433  502.209     498.48   506.11   499.244  512.53 
 514.641  516.13   511.533  518.931     511.176  519.687  520.316  519.163
 512.068  517.94   509.921  503.95      507.119  516.949  512.324  520.267
 485.999  502.84   491.221  500.771  …  492.548  501.295  498.827  501.975
 500.929  503.421  496.193  500.245     491.89   505.216  497.835  512.739
 510.457  520.259  509.842  511.498     507.358  519.876  511.89   519.129
 518.47   526.671  508.699  523.33      523.943  528.418  522.436  529.202
 493.97   508.688  495.181  497.276     495.346  508.618  505.535  512.053
 507.439  514.698  503.336  511.782  …  516.755  511.987  516.746  519.88 
 493.011  500.946  498.185  502.382     502.943  514.524  503.265  509.855
 494.975  510.644  498.354  505.626     503.671  501.076  499.159  512.6

In [11]:
@test isapprox(parallel_result,serial_result)

Test Passed

In [12]:
@btime serial_result = serial_mult(a,b,N)

  248.741 ms (4 allocations: 48.00 MiB)


2048×2048 Array{Float32,2}:
 500.211  505.209  494.247  500.829  …  497.685  511.218  510.327  511.967
 518.749  514.911  514.287  519.131     517.602  526.828  521.151  526.347
 497.019  502.988  497.434  502.209     498.48   506.109  499.244  512.529
 514.641  516.13   511.534  518.931     511.175  519.687  520.316  519.163
 512.068  517.94   509.921  503.95      507.12   516.948  512.325  520.267
 485.998  502.84   491.221  500.771  …  492.548  501.295  498.827  501.975
 500.93   503.422  496.193  500.245     491.89   505.215  497.834  512.739
 510.457  520.26   509.842  511.498     507.358  519.877  511.89   519.129
 518.471  526.671  508.699  523.33      523.943  528.418  522.436  529.202
 493.97   508.688  495.181  497.276     495.346  508.618  505.536  512.053
 507.439  514.697  503.336  511.782  …  516.754  511.987  516.746  519.88 
 493.011  500.946  498.185  502.382     502.943  514.524  503.264  509.855
 494.976  510.644  498.355  505.626     503.671  501.076  499.159  512.6

In [13]:
@btime @cuda blocks = (Int(ceil(N/32)),Int(ceil(N/32))) threads = (32,32) parallel_mult(d_a,d_b,d_output,N)

  14.033 μs (71 allocations: 2.58 KiB)


## The parallel code is about 10^4 times faster than the serial code